# Assignment 1

## Initialization

In [ ]:
# pip install adabelief-tf==0.2.0
# pip install imblearn

In [ ]:
import pandas as pd
import numpy as np
import cv2
data_path = 'dataForClass.csv' # training data
image_size=(48,48)

def load_data(data_path):
    data = pd.read_csv(data_path)
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(width, height)
        face = cv2.resize(face.astype('uint8'),image_size)
        faces.append(face.astype('float32'))
    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)

    emotions = pd.get_dummies(data['emotion']).values
    return faces, emotions
    
faces, emotions = load_data(data_path)

### Understanding of Data

In [ ]:
# view number of train images per class
data = pd.read_csv(data_path)
d = data['pixels'].count()
data.groupby('emotion').describe()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
ind = 23650
f = faces[ind, :,:,0]
# print('Emotion Labels', emotions[ind,:])
plt.imshow(f, cmap='gray')
emotion_dict =  {0: "Angry", 1: "Disgusted" ,2:"Worried", 3:"Happy", 4:"Sad", 5: "Terrified"}

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,10))
emotion_idx = np.argmax(emotions, axis=1)
# Show first 25 training images below
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(faces[i, :, :, 0], cmap='gray')
    plt.xlabel(emotion_dict[emotion_idx[i]])

## Fixing Imbalanced Data (By oversampling minority data)

In [ ]:
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
oversample = SMOTE()
label_emotion = data['emotion']
y = LabelEncoder().fit_transform(label_emotion)
tmp = faces.reshape((faces.shape[0], 48*48))
x, y = oversample.fit_resample(tmp, y)
tmp_x = x.reshape((x.shape[0], 48, 48, 1))
tmp_y = pd.get_dummies(y).values

In [ ]:
from collections import Counter
from matplotlib import pyplot
counter = Counter(y)
for k,v in counter.items():
    per = v / len(y) * 100
    print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
pyplot.bar(counter.keys(), counter.values())
pyplot.show()

## Training

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D, Conv2D, UpSampling2D, MaxPool2D, AveragePooling2D, Flatten, Dropout, BatchNormalization, MaxPooling2D, Activation, LSTM 
from tensorflow.keras.layers import SeparableConv2D, Reshape, Input, Lambda, Concatenate
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import L2
from adabelief_tf import AdaBeliefOptimizer

In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(UpSampling2D(size=(2, 2), interpolation='nearest'))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(1048, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1048, activation='relu'))
model.add(Dense(6, activation='softmax'))

model.summary()


In [ ]:
train_batchsize = 128
validation_batchsize = 128

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    validation_split=0.2)

train_generator = datagen.flow(
    x = tmp_x,
    y = tmp_y,
    shuffle = True,
    batch_size=train_batchsize,
    subset='training') # set as training data

validation_generator = datagen.flow(
    x = tmp_x,
    y = tmp_y,
    batch_size=validation_batchsize,
    subset='validation') # set as validation data

optimizer = Nadam(learning_rate=1e-3, epsilon=1e-7)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['acc'])


In [ ]:
# save the model
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='Brendon.h5', mode='max', monitor='val_acc', verbose=2, save_best_only=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=6, 
                                            verbose=1, 
                                            factor=0.35, 
                                            min_lr=0.000003)

callbacks_list = [checkpoint, learning_rate_reduction]

epochs = 250

model.fit(
      train_generator,
      steps_per_epoch= train_generator.n/train_generator.batch_size ,
      epochs=epochs,
      validation_data=validation_generator,
      validation_steps=validation_generator.n/validation_generator.batch_size,
      verbose=1,
      callbacks=callbacks_list)

## Saving the model

In [ ]:
# save your model and weight (only submit best model)
model_json = model.to_json()
with open("Brendon.json", "w") as json_file:
    json_file.write(model_json)